In [ ]:
#NEW KERNEL

In [ ]:
# importing the tools required for the Poisson regression model
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from scipy.stats import poisson,skellam

In [ ]:
# Set teams here
home_team='Arsenal'
away_team='Fulham'

In [ ]:
epl = pd.read_csv(r"C:\Users\blake\OneDrive\Documents\Pl_2223 xG.csv",encoding='latin1')
ep = epl[['Home','Away','xGh','xGa']]
epl = epl.rename(columns={'xGh': 'HomeGoals', 'xGa': 'AwayGoals'})
epl.head()

In [ ]:
print(epl)

In [ ]:
goal_model_data = pd.concat([epl[['Home','Away','HomeGoals']].assign(home=1).rename(
            columns={'Home':'team', 'Away':'opponent','HomeGoals':'goals'}),
           epl[['Away','Home','AwayGoals']].assign(home=0).rename(
            columns={'Away':'team', 'Home':'opponent','AwayGoals':'goals'})])

poisson_model = smf.glm(formula="goals ~ home + team + opponent", data=goal_model_data,
                        family=sm.families.Poisson()).fit()
poisson_model.summary()

In [ ]:
#Predict for Home Team vs. Away Team
home_score_rate=poisson_model.predict(pd.DataFrame(data={'team': home_team, 'opponent': away_team,
                                       'home':1},index=[1]))
away_score_rate=poisson_model.predict(pd.DataFrame(data={'team': away_team, 'opponent': home_team,
                                       'home':0},index=[1]))
print(home_team + ' against ' + away_team + ' xG: ' + str(home_score_rate))
print(away_team + ' against ' + home_team + ' xG: ' + str(away_score_rate))

#Lets just get a result
home_goals=np.random.poisson(home_score_rate)
away_goals=np.random.poisson(away_score_rate)
print(home_team + ': ' + str(home_goals[0]))
print(away_team + ': '  + str(away_goals[0]))

In [ ]:
# Code to caluclate the goals for the match.
def simulate_match(foot_model, homeTeam, awayTeam, max_goals=10):
    home_goals_avg = foot_model.predict(pd.DataFrame(data={'team': homeTeam,
                                                           'opponent': awayTeam, 'home': 1},
                                                     index=[1])).values[0]
    away_goals_avg = foot_model.predict(pd.DataFrame(data={'team': awayTeam,
                                                           'opponent': homeTeam, 'home': 0},
                                                     index=[1])).values[0]
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals + 1)] for team_avg in
                 [home_goals_avg, away_goals_avg]]
    return (np.outer(np.array(team_pred[0]), np.array(team_pred[1])))

#Fill in the matrix
max_goals=5
score_matrix=simulate_match(poisson_model, home_team, away_team,max_goals)

fig=plt.figure()
ax=fig.add_subplot(1,1,1)
pos=ax.imshow(score_matrix, extent=[-0.5,max_goals+0.5,-0.5,max_goals+0.5], aspect='auto',cmap=plt.cm.Reds)
fig.colorbar(pos, ax=ax)
ax.set_title('Probability of outcome')
plt.xlim((-0.5,5.5))
plt.ylim((-0.5,5.5))
plt.tight_layout()
ax.set_xlabel('Goals scored by ' + home_team)
ax.set_ylabel('Goals scored by ' + away_team)
plt.show()

#Home, draw, away probabilities
homewin=np.sum(np.tril(score_matrix, -1))
draw=np.sum(np.diag(score_matrix))
awaywin=np.sum(np.triu(score_matrix, 1))

In [ ]:
print(home_team,"Win percentage = ", round(homewin*100, 2),"%")

print("Draw percentage = ",round(draw*100,2), "%")


print(away_team, "Win percentage = ", round(awaywin*100, 2),"%")